# EE 120 Lab 2: Fourier Series for Audio Compression

**Signals and Systems** at UC Berkeley

Fall 2021 - BETA

# Table of Contents
* [Background](#bg)
* [Q1: The Fourier Series Approximation](#q1)
    * [Q1.a: Fourier Series Coefficients](#q1a)
    * [Q1.b: Most Significant Coefficients](#q1b)
    * [Q1.c: Signal Reconstruction](#q1c)
* [Q2: Application to Audio Compression](#q2)
    * [Q1.a: Chopping Up Our Signal](#q2a)
    * [Q1.b: Compressing Audio](#q2b)

<a id='bg'></a>
# Background: Digital Audio Processing

### File Types
Audio signals are perhaps one of the most used types of 1D digital signals. They are commonly stored on your computer as .wav or `.mp3` files. The `.wav` file format will actually store the raw discrete-time audio samples in the file. This is the format that we will be using for the duration of the lab.  

### Sampling Rate
Perhaps the most important metric in describing digital audio signals is the sampling rate. The sample rate (in Hz) is a measure of the number of samples of a signal that you send per second. 

Suppose that we want to send $N$ samples at a sampling rate of $F_s$ Hz. The total duration of the signal in seconds is $T = N T_s$, where $T_s$ is the spacing, in seconds, between two consecutive samples. It is true that $T_s = \frac{1}{F_s}$.

As mentioned before, we will be dealing with `.wav` files when creating audio signals. So, let us import some libraries to help us with `.wav` files and audio processing (you may need to use conda or pip to install any missing packages):

In [ ]:
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from fast_methods_120 import *
from IPython.display import Audio

<a id='q1'></a>
# Q1: Fourier Series Approximation

## DTFS Review

In homework 4, we explored the trigonometric form of the DTFS. More specifically, we considered some real $N$-periodic discrete-time signal $x[n]$. The synthesis equation gives us
$$
x[n] = \sum_{k = 0}^{N-1} a_k e^{j2\pi nk / N}
$$
and the analysis equation gives us 
$$
a_k = \frac{1}{N} \sum_{n = 0}^{N-1} x[n] e^{-j 2 \pi n k / N}
$$
where we can split up the Fourier coefficients into real and imaginary parts: $a_k = b_k + j c_k$. That is, the real part of $a_k$ is $b_k$ and the imaginary part of $a_k$ is $c_k$. Since $x[n]$ is real, we know that
$$
a_k^* = a_{-k} = a_{N - k}
$$

We then showed that the trigonometric Fourier series of $x[n]$ is 
$$
x[n] = a_0 + 2\big(\sum_{k=1}^{\frac{N-1}{2}}\mathbb{R}\text{e} \{a_k\} \cos(2\pi nk / N) - \mathbb{I}\text{m} \{a_k\} \sin(2\pi nk / N) \big),
$$
where $N$ is odd.

Throughout the lab, we will compute $a_k$ via the synthesis equation, but we will represent $x[n]$ using its trigonometric form. Note that we only need to compute $a_k$ for positive values of $k$ because our audio signal is real, implying $a_{-k} = a^*_k$.

<a id='q1a'></a>
## Q1a: Fourier Series Coefficients

We would now like to write a function that returns all the Fourier series coefficients of some signal $x[n]$. This function will be called `get_fourier_coeffs(x)` where `N = len(x)`. 

*Side Note*: You may have noticed that the discrete-time Fourier series equations were introduced to analyze periodic discrete-time signals. However, the input array `x` is of finite length `len(x)` (i.e. not periodic). So how can we expect to use the Fourier series formulas? The answer is to just assume that `x` is just one period of an `N` periodic signal, where `N = len(x)`. This variation of applying the DTFS formula to some time interval of an aperiodic signal is known as the DFT (discrete Fourier transform), and is widely used in many applications. However, for the remainder of this lab, we will continue to refer to it as the DTFS. 

### Your Job
Fill in the function `get_fourier_coeffs(x)`.   
`x` (`np.array`) is a discrete-time real signal.   
`returns` (`np.array`) $[a_0, ..., a_{N//2}]$ (i.e. the first $1 + N//2$ Fourier series coefficients of `x`). 

**IMPORTANT:** We are dealing with real signals, so you should not return all coefficients. Only the first $1 + N//2$ coefficients. (N//2 means N *integer division* by 2 (rounded down). 3//2 = 1, 4//2 = 2).

The first coefficient should be $a_0$ and the last coefficient should be $a_{N//2}$. 

*Tip*: Python supports imaginary numbers by appending j after a number. Ex: 1+1j gives a complex number with both the real and imaginary component equal to 1.


*Examples*   
```
get_fourier_coeffs([1, 0, 0, 0]) 
> [0.25, 0.25, 0.25]
```
```
get_fourier_coeffs([1, 1, 1, 1]) 
> [0.25, 0, 0]
```


In [ ]:
def get_fourier_coeffs(x):
    N = len(x)
    ak = np.zeros(N//2 + 1, dtype=np.complex128)
    # ---------- YOUR CODE START ----------
    
    # ----------- YOUR CODE END -----------
    return ak

**Make sure you see the message** `All Tests Passed!` **after running the cell below:**

In [ ]:
test_get_fourier_coeffs(get_fourier_coeffs)

<a id='q1b'></a>
## Q1b: Most Significant Coefficients

Suppose that we have a signal $x[n]$ with period $N$. This signal has Fourier series coefficients $a_0, a_1, ..., a_{N-1}$. Not all of these coefficients have equal magnitude. Some coefficients may be larger in magnitude, and are thereby *more important* - they carry more information and make up more of the actual signal than other coefficients. 

We say that the $M$ most significant Fourier series coefficients are the $M$ coefficients with the largest magnitude. So, for example, suppose we have 
$a_0 = 1, a_2 = 0.5j, a_3 = -10 - 15j, a_4 = 0.3$ for some signal of period $N = 4$. The $M = 2$ most significant coefficients are $a_3$ and $a_0$ (in that order). That is because those two coefficients have the largest magnitude of the entire set of coefficients.

### Your Job

Fill in the function for `M_most_sig(a_k, M)`:   
`a_k` (`np.array`) the array of Fourier series coefficients, starting with $a_0$.    
`M` (`int`) Represents the number of coefficients we'd like returned (i.e. the number of significant coefficients).    
This will return two arrays:   
`returns` (`np.array`) a list of the `M` most important (largest magnitude) indices (values of $k$).    
`returns` (`np.array`) a list of the `M` most important (largest magnitude) values (the actual $a_k$). 

**Important:** This only returns *non-zero* coefficients and should be ordered from largest to smallest magnitude.

So, suppose that `ak = [0, 0, 0, 100, 200]` and `M = 2`. Then, we would like to return `[4, 3]`, `[200, 100]`. The first list says that $a_4$ and $a_3$ are the most important coefficients, in that order. The second list specifies the value of $a_4$ and $a_3$, with $a_4$ first because $a_4 > a_3$.

Similarly, suppose that for the same `ak`, we have `M = 3`. Then, we would like to return `[4, 3]`, `[200, 100]`. Notice that there are only two values to return, since the next most significant coefficient is 0.

*Examples*   
```
M_most_sig(np.zeros(10), M=2) 
> [], []
```
```
x1 = np.array([0, 10, 12, 0])
M_most_sig(x1, M=2) 
> [2, 1], [12, 10]
```
```
M_most_sig(x1, M=1) 
> [2], [12]
```
```
x2 = np.array([1 -100j, 0.1 - 0.2j, 0, 0, 200j])
M_most_sig(x2, M=5) 
> [4, 0, 1], [200j, 1 - 100j, 0.1 - 0.2j]
```
```
M_most_sig(x2, M=2) 
> [4, 0], [200j, 1 - 100j]
```

In [ ]:
def M_most_sig(ak, M):
    k_sig = []
    ak_sig = []
    ak = ak.copy()
    # ---------- YOUR CODE START ----------
    
    # ----------- YOUR CODE END -----------
    return np.array(k_sig), np.array(ak_sig)


**Make sure you see the message** `All Tests Passed!` **after running the cell below:**

In [ ]:
test_M_most_sig(M_most_sig)

<a id='q1c'></a>
## Q1c: Signal Reconstruction

So far we have a function from part (a) that finds the Fourier series coefficients $a_k$ (or rather, only half of them, since we are dealing with real signals) of some real signal $x[n]$. Then, from part (b), we have a function that will pick out the $M$ most significant Fourier series coefficients from the entire set of $a_k$. 

Even though we need all of the coefficients ak to fully represent $x[n]$, a compressed version of $x[n]$ that is made up of the top $M$ coefficients may contain enough information to reconstruct something reasonable.

If we try and reconstruct a **real** signal - call it $x_r[n]$ - off of only $M$ Fourier series coefficients of $x[n]$, what would it look like? Let us find out! 

Suppose our reconstruced signal has the form
$$
x_r[n] = r_0 + 2\big(\sum_{k=1}^{\frac{N-1}{2}}b_k \cos(2\pi nk / N) - c_k\sin(2\pi nk / N) \big),
$$
where $r_k = b_k + j c_k$. The $b_k$ are the real components of the $r_k$ and the $c_k$ are the imaginary components of $r_k$.

Our function will compute the $x_r[n]$ above by setting the $r_k$ according to the function inputs. The function will take in two lists and a integer representing the signal length `N`. The first list, `k_sig`, is a list of the non-zero indices of $r_k$. The second list, `rk_sig`, is a list of the non-zero *values* of $r_k$. Then, using the formula above, the function will return $x_r[n]$.

So, if `k_sig = [2, 4]` and `r_sig = [10j, -4]`, then we know that $r_2 = 10j, r_4 = -4$, and all other $k \ne 2, k \ne 4$ have give $r_k = 0$.

### Your Job

Fill in the function `recon(k_sig, rk_sig, N)`:    
`k_sig` is the list of the non-zero indices of $r_k$.  
`rk_sig` is the list of the non-zero values of $r_k$.   
`N` is the length/period of the signal that we want to reconstruct.    
`returns` (`np.array`) the new Fourier series coefficients $r_k$. 

In [ ]:
def recon(k_sig, rk_sig, N):
    assert len(k_sig) == len(rk_sig)
    xr = np.zeros(N)
    for n in range(N):
        for i in range(len(k_sig)):
            # index k
            k = k_sig[i]
            
            # coefficient rk
            rk = rk_sig[i]
            
            if k == 0:
            # If we have a_0, we add it differently
                xr[n] += np.real(rk)
            else:
            # Case where k > 0. This is the summation in the formula above
                bk = np.real(rk)
                ck = np.imag(rk)
                # ---------- YOUR CODE START ----------
                xr[n] += ?
                # ----------- YOUR CODE END -----------
    return xr

### Testing Your Function and Putting It All Together

From part (a), we are able to find the Fourier coefficients of a signal $x[n]$. In part (b), we picked out the $M$ most significant Fourier coefficients of the signal $x[n]$. Finally, in part (c) we will reconstruct a new signal $x_r[n]$ off of only $M$ coefficients.

Now we will put these three parts together into one function, `M_approximation`:

In [ ]:
def M_approximation(M, x):
    # Get Fourier series coefficients ak
    ak = get_fourier_coeffs(x)
    # Find only the top M most significant coefficients
    k_sig, rk_sig = M_most_sig(ak, M)
    # Use the M most significant ak to construct xr
    xr = recon(k_sig, rk_sig, len(x))
    return xr

In [ ]:
# Define a pulse of length N
N = 1001
x_pulse = np.zeros(N)
x_pulse[N//4:3 * N//4] = 1

# Plot out the pulse
plt.plot(x_pulse)
plt.title('Pulse')

Above we have constructed a pulse called, `x_pulse`. To test that our `recon` function, and really the whole pipeline is working correctly, we will verify that the  M_approximation of a pulse looks good!

**Make sure you see the message** `Reconstruction Test Passed!` **after running the cell below:**

In [ ]:
xr = M_approximation(20, x_pulse)
xr_expected = np.load('pulse_expected.npy')
if np.linalg.norm(xr_expected - xr) > 1e-6:
    print("Pulse Test Failed. Make sure that 'recon' is working properly.")
else:
    print('Reconstruction Test Passed!')

Now that we know that our reconstruction above has passed, let us view what this looks like for different values of $M$. Remember, $M$ is just how many Fourier coefficients we are keeping from our original set.

In [ ]:
Ms = [5, 10, 20, 50, 100]
plt.figure(figsize=(14, 10))
for i, M in enumerate(Ms):
    xr = M_approximation(M, x_pulse)
    plt.subplot(len(Ms), 1, i+1)
    plt.plot(x_pulse)
    plt.plot(xr, label = f'M = {M}')
    plt.legend()

<a id='q2'></a>
# Q2: Application to Audio Compression


In order for everything in this question to work, we are relying on your implementation of the function `M_most_sig`. **If `M_most_sig` is not passing the test cases, please go back and make sure that it does**.

Most audio signals are sampled at around 44.1kHz. That is, in one second, we have 44,100 samples. Suppose you download a 3 minute song. This is 180 seconds of audio sampled at 44.1kHz. This is close to a whopping 8 million samples! At 2 signed bytes per sample, storing this song in its raw form would take about 16 MB. Today, a standard 3-minutes MP3 file takes up about 3 MB. It turns out that using what we know about the discrete-time Fourier series, we can create a primitive compression scheme to significantly reduce the size of audio files. 

We will start by loading in a 22 second audio clip of the song 'Dancing Queen' by ABBA. An audio signal is nothing but a discrete-time signal. In the cell down below, print out the length of the signal and the sample rate



### **Warning:** This might be loud, so turn down your volume when playing the audio.

In [ ]:
fs, song = wavfile.read('dancing_queen_clip_float.wav')
Audio(song, rate=fs)

In [ ]:
# Printing the length of the song in samples
print('There are', len(song), 'samples in this sound clip.')
print(f'The sampling rate is {fs} samples per second.')

We have about a million samples in our audio clip. The goal of the next parts will be to compress this down to a much smaller amount of values. 

In Question 1, we developed a technique of choosing the largest $M$ Fourier series coefficients of some discrete-time signal $x[n]$. That is, instead of representing $x[n]$ by all $N$ samples, we represented it with two lists of length $M$. So, the total size (in elements) would be $2 M$ as apposed to $N$ elements to represent $x[n]$. If we choose $M$ to be small enough, our memory savings will be quite significant. 

But how small can we make $M$? If we make $M$ too small, it won't represent our true signal $x[n]$ very well. But if we make $M$ too large, then we will not acheive a very high compression. It turns our that audio signals are fairly compressible. That means that we can get away with choosing a relatively small value of $M$ while retaining most of our audio's *quality*.

However, as we saw above, our audio signal $x[n]$ has nearly one million samples in it. We need to break it up into smaller peices first.

<a id='q2a'></a>
## Q2a: Chopping Up Our Signal

To make our audio signal more managable, let us chop it up into smaller 'bite size' signals. 

### Your Job
Fill in the function `chop(x, block_size)`:   
`x` (`np.array`) is a real discrete-time signal of length/period `N = len(x)`.    
`block_size` (`int`) is the size of the individual blocks in our signal `x`. In general, we will write our signal `x` as `N/block_size` blocks of size `block_size`.   
`returns:` (`list`) list of all of the blocks of size `block_size` in our signal `x`.   

*Examples:*   
```
x1 = np.array([1,2,3,4,5,6,7,8])
block_size = 4
chop(x, block_size)
> [np.array([1,2,3,4]), np.array([5,6,7,8])]
```
```
x2 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
block_size = 5
chop(x1, block_size)
> [np.array([1,2,3,4,5]), np.array([6,7,8,9,10]), np.array([11,12,13,14,15])]
```

In [ ]:
def chop(x, block_size):
    # add zeros to x so that it evenly divides into block_size chunks
    Nz = int(np.ceil(len(x) / block_size) * block_size) - len(x)
    x = np.concatenate((x, np.zeros(Nz)))
    
    # Length of x and blocks to return
    N = len(x)
    blocks = []
    # ---------- YOUR CODE START ----------
    
    # ----------- YOUR CODE END -----------
    return blocks


**Make sure you see the message** `All Test Passed!` **after running the cell below:**

In [ ]:
test_chop(chop)

<a id='q2b'></a>
## Q2b: Compressing Audio

We will now modify the function `M_approximation` to work operate on each block in our song. In addition, the functions that you wrote `get_fourier_coeffs` and `recon`, as amazing as they were, are simply not fast enough. For reasons that we will explore later in the class, there is a much faster way of computing the Fourier series coefficients of a signal. For that reason, we use the staff solution, which utilizes the *FFT* (fast Fourier transform - more on this later in the course). However, we will still be using your implementation of `M_most_sig`! 

### Your Job

Fill in the function `M_approximation_blocks`:   
`x` (`np.array`) signal that we would like to compress.   
`M` (`int`) Fourier series approximation integer.      
`block_size` (`int`) size of each block we'd like to chop up `x` into.   
`returns:` (`np.array`) reconstructed signal.   

In [ ]:
def M_approximation_blocks(x, M, block_size):
    # Chop x into many blocks of size block_size
    blocks = chop(x, block_size)
    
    # Reconstructed signal to return
    xr = np.zeros(len(blocks) * block_size)
        
    # Iterate through each block and apply the M approximation on them
    for i in range(len(blocks)):
        # Current block
        block = blocks[i]
        
        # Get Fourier series coefficients ak. Using staff solution for speed
        ak = fast_get_fourier_coeffs(block)
        
        # Find only the top M most significant coefficients. Uses your solution
        k_sig, rk_sig = M_most_sig(ak, M)
                
        # Use the M most significant ak to construct xr. Using staff solution for speed
        xr_block = fast_recon(k_sig, rk_sig, block_size)
        
        # Combine xr_block into xr 
        # ---------- YOUR CODE START ----------
        
        # ----------- YOUR CODE END -----------
        
    return xr

In order to verify the functionality of the function above, try out the function for different values of `M` and `block_size` to see how the audio quality changes. 

We define compression ratio as:
$$
\text{Compression Ratio} = \frac{\text{Original Size}}{\text{New Size}} = \frac{\text{block_size}}{\text{2M}}
$$

**Warning:** This might be loud.

In [ ]:
%%time
block_size = 2 ** 8
comp_ratio = 5
M = block_size // (2 * comp_ratio)
song_recon = M_approximation_blocks(song, M=M, block_size=block_size)
print(r'Actual Compression Ratio =', block_size / (2 * M))

In [ ]:
Audio(song_recon, rate=fs)

Notice that we all printed out the Wall time, which is a measure of how long the program takes to run. Comment on what values of `M` and `block_size` work best. Your goal should be to maximize the Compression Ratio but minimize the Wall time. There is no right answer - feel free to try out as many different pairs as you would like.

Note that there are other compression techniques out there! This is just a start on one of them. Many other things can be compounded on to further reduce the size of the audio file - if you're interested, some keywords of interest might be temporal masking and bit rate & bit depth reduction. These involve some psychoacoustics and incorporate theory on how humans perceive sound. If things like this are cool to you, take EE 123!

**YOUR COMMENT HERE**